In [1]:
%matplotlib inline

import nengo
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

import nengo_dl

# keras uses the global random seeds, so we set those here to
# ensure the example is reproducible
seed = 0
np.random.seed(seed)
tf.random.set_seed(seed)

In [2]:
X = np.array([[0,0],[0,1],[1,0],[1,1]])
y = np.array([0,1,1,0])

In [87]:
X.shape

(4, 2)

In [88]:
y.shape

(4,)

In [89]:
set(y)

{0, 1}

In [170]:
np.ones(X.shape[1])

array([1., 1.])

In [171]:
X.shape[1]

2

In [226]:
with nengo.Network(seed=0) as net:
    # set up some default parameters to match the Keras defaults
    net.config[nengo.Ensemble].gain = nengo.dists.Choice([1])
    net.config[nengo.Ensemble].bias = nengo.dists.Choice([0])
    net.config[nengo.Connection].synapse = None
    net.config[nengo.Connection].transform = nengo_dl.dists.Glorot()

    # input node, same as before
    inp = nengo.Node(output=np.ones(X.shape[1]))

    # add the first dense layer
    hidden = nengo.Ensemble(8, 1, neuron_type=nengo.RectifiedLinear()).neurons
    nengo.Connection(inp, hidden)
    '''hidden2 = nengo.Ensemble(1, 1, neuron_type=nengo.Sigmoid()).neurons
    nengo.Connection(hidden, hidden2)'''

    # add the linear output layer (using nengo.Node since there is
    # no nonlinearity)
    #out = nengo.Node(nengo.Sigmoid, size_in=1)
    out = nengo_dl.Layer(
        tf.keras.layers.Dense(units=1, activation=tf.nn.sigmoid))(hidden)
    #out = nengo.Ensemble(1, 1, neuron_type=nengo.Sigmoid()).neurons
    nengo.Connection(hidden, out)

    # add a probe to collect output
    out_p = nengo.Probe(out)

In [227]:
#from sklearn.model_selection import train_test_split
X_snn = X[:, None, :]
y_snn = y[:, None, None]
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [228]:
X_snn.shape

(4, 1, 2)

In [229]:
y_snn.shape

(4, 1, 1)

In [230]:
with net:
    nengo_dl.configure_settings(stateful=False, use_loop=False)

with nengo_dl.Simulator(net, minibatch_size=4) as sim:
    sim.compile(optimizer=tf.optimizers.SGD(lr=0.1),
                loss=tf.keras.losses.binary_crossentropy,
                metrics=["accuracy"])
    sim.fit(X_snn, y_snn, epochs=1000, verbose=0)

    print("Training accuracy:", sim.evaluate(
        X_snn, y_snn, verbose=0)["probe_accuracy"])
    
    sim.save_params("./XOR_snn")

Build finished in 0:00:00                                                      
Optimization finished in 0:00:00                                               


C:\ProgramData\Anaconda3\lib\site-packages\nengo\neurons.py:403: RuntimeWarning: divide by zero encountered in log1p
  1.0 / (gain * (intercepts - 1) - 1)))


Training accuracy: 0.5


In [225]:
from sklearn.metrics import f1_score, accuracy_score
with nengo_dl.Simulator(net, minibatch_size=4) as sim2:
    sim2.load_params("./XOR_snn")
    # call compile and evaluate, as we did with the Keras model
    sim2.compile(optimizer=SGD(lr=0.1),
                loss=tf.keras.losses.binary_crossentropy,
                metrics=["accuracy"])
    print("Test accuracy:", sim2.evaluate(
        X_snn, y_snn, verbose=0)["probe_accuracy"])
    predictions = sim2.predict(X_snn)[out_p]
    y_pred = []
    for p in predictions:
        y_pred.append(p[0][0])
    #print("Test loss: ", sim2.evaluate(X_snn, y_snn, verbose=0)['loss'])
    #print("Test accuracy: ", accuracy_score(y_snn.flatten(), y_pred))
    #print("Test F1 Score: ", f1_score(y_snn.flatten(), y_pred, average='micro'))
    print(y_pred)

Build finished in 0:00:00                                                      
Optimization finished in 0:00:00                                               
Test accuracy: 1.0
[0.0027453303, 0.9987272, 0.9987707, 0.0009176433]


In [36]:
X.shape

(4, 2)

In [37]:
y.shape

(4,)

## Keras Model

In [108]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
model = Sequential()
model.add(Dense(8, input_shape=(X.shape[1],), activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer=SGD(lr=0.1),
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.fit(X, y, epochs=1000, verbose=0)
model.predict_proba(X)

array([[0.05726558],
       [0.9900018 ],
       [0.9899245 ],
       [0.00952592]], dtype=float32)

In [35]:
model_weights = "keras_weights"
model.save_weights(model_weights)

In [39]:
class KerasWrapper(tf.keras.layers.Layer):
    def __init__(self, keras_model):
        super().__init__()

        self.model = keras_model

    def build(self, input_shapes):
        super().build(input_shapes)

        # we use clone_model to re-build the model
        # within the TensorNode context
        self.model = tf.keras.models.clone_model(self.model)

        # load the weights we saved above
        self.model.load_weights(model_weights)

    def call(self, inputs):
        # apply the model to the inputs
        return self.model(inputs)

In [47]:
with nengo.Network() as netKeras:
    # create a normal input node to feed in our test image.
    # the `np.ones` array is a placeholder, these
    # values will be replaced with the Fashion MNIST images
    # when we run the Simulator.
    input_node = nengo.Node(output=np.ones(X.shape[1]))

    # create an instance of the custom layer class we created,
    # passing it the Keras model
    layer = KerasWrapper(model)

    # create a TensorNode and pass it the new layer
    keras_node = nengo_dl.TensorNode(
        layer,
        shape_in=(X.shape[1],),  # shape of input (the flattened images)
        shape_out=(1,),  # shape of output (# of classes)
        pass_time=False,  # this node doesn't require time as input
    )

    # connect up our input to our keras node
    nengo.Connection(input_node, keras_node, synapse=None)

    # add a probe to collect output of keras node
    keras_p = nengo.Probe(keras_node)

In [64]:
with nengo_dl.Simulator(netKeras, minibatch_size=4) as sim:
    # call compile and evaluate, as we did with the Keras model
    sim.compile(optimizer=SGD(lr=0.1),
                loss=tf.keras.losses.binary_crossentropy,
                metrics=["accuracy"])
    print("Test accuracy:", sim.evaluate(
        X_snn, y_snn, verbose=0)["probe_accuracy"])
    predictions = sim.predict(X_snn)[keras_p]
    y_pred = []
    for p in predictions:
        y_pred.append(p[0][0])
    print("Test F1 Score: ", f1_score(y_snn.flatten(), list(map(round, y_pred)), average='micro'))
    print(y_pred)

Build finished in 0:00:00                                                      
Optimization finished in 0:00:00                                               
Test accuracy: 1.0
Test F1 Score:  1.0
[0.040897667, 0.9884031, 0.9880152, 0.040954262]


## Keras into Nengo-DL

In [177]:
out = nengo_dl.Layer(
        tf.keras.layers.Dense(units=1, activation=tf.nn.sigmoid))
out

In [178]:
out.layer_func

In [151]:
nengo.Sigmoid

nengo.neurons.Sigmoid

In [117]:
with nengo.Network(seed=seed) as net:
    # input node, same as before
    inp = nengo.Node(output=np.ones(X.shape[1]))

    # add the Dense layers, as in the Keras model
    hidden = nengo_dl.Layer(
        tf.keras.layers.Dense(units=8, activation=tf.nn.relu))(inp)
    out = nengo_dl.Layer(
        tf.keras.layers.Dense(units=1, activation=tf.nn.sigmoid))(hidden)

    # add a probe to collect output
    out_p = nengo.Probe(out)

In [122]:
with net:
    nengo_dl.configure_settings(stateful=False, use_loop=False)

with nengo_dl.Simulator(net, minibatch_size=4) as sim:
    sim.compile(optimizer=tf.optimizers.SGD(lr=0.1),
                loss=tf.keras.losses.binary_crossentropy,
                metrics=["accuracy"])
    sim.fit(X_snn, y_snn, epochs=1000, verbose=0)

    print("Test accuracy:", sim.evaluate(
        X_snn, y_snn, verbose=0)["probe_accuracy"])
    predictions = sim.predict(X_snn)[out_p]
    y_pred = []
    for p in predictions:
        y_pred.append(p[0][0])

Build finished in 0:00:00                                                      
Optimization finished in 0:00:00                                               
Test accuracy: 1.0


In [123]:
y_pred

[0.09212786, 0.98423856, 0.9712148, 0.016425222]